In [2]:
# Imports
import pandas as pd
import numpy as np
# from skmultiflow.drift_detection.adwin import ADWIN
from hyppo.ksample import KSample

import os

import time

In [11]:
# used as input to drift_detection methods
def xnor(x1, x2):
    '''
    '''
    return ~np.logical_xor(x1, x2)

In [3]:
idx=pd.IndexSlice

In [4]:
data_dir="E:\Data\HIDENIC_EXTRACT_OUTPUT_DIR\POP_SIZE_0\ITEMID_REP\hospital-style_trainHospitals_UPMCPUH_raw_Simple_seed_0_target=mort_icu.h5"

site_info = pd.read_pickle("E:/Data/HIDENIC_EXTRACT_OUTPUT_DIR/POP_SIZE_0/ITEMID_REP/site_info.pkl")

subject_info = pd.read_hdf("E:/Data/HIDENIC_EXTRACT_OUTPUT_DIR/POP_SIZE_0/ITEMID_REP/filtered_data.h5", "label_df")

subject_info=subject_info.join(site_info.set_index(['subject_id', 'hadm_id'])[['hospital', 'icu_category']],
       on=['subject_id', 'hadm_id'])

In [ ]:
hospitals = set(site_info["hospital"].unique()) - set(["UPMCPUH"])

In [3]:
X1 = pd.read_hdf(data_dir, key="X_test_UPMCPUH").drop('time', axis=1, level=2).values

In [210]:
hospitals

{'UPMCBED',
 'UPMCEAS',
 'UPMCHAM',
 'UPMCHZN',
 'UPMCMCK',
 'UPMCMER',
 'UPMCMWH',
 'UPMCNOR',
 'UPMCPAS',
 'UPMCSHY',
 'UPMCSMH'}

In [59]:
for hosp in hospitals:
    X2 = pd.read_hdf(data_dir, key="X_test_" + hosp).drop('time', axis=1, level=2).values
    stat, pvalue = KSample("Dcorr").test(X1, X2)
    print('{}: stat={:0.2f}, pval={:0.3f}'.format(hosp, stat, pvalue))
    

UPMCMWH: stat=0.21, pval=0.000
UPMCMCK: stat=0.04, pval=0.000
UPMCPAS: stat=0.32, pval=0.000
UPMCMER: stat=0.30, pval=0.000
UPMCHAM: stat=0.51, pval=0.000
UPMCHZN: stat=0.52, pval=0.000
UPMCBED: stat=0.45, pval=0.000
UPMCSMH: stat=0.47, pval=0.000
UPMCNOR: stat=0.41, pval=0.000
UPMCEAS: stat=0.46, pval=0.000
UPMCSHY: stat=0.26, pval=0.000


In [10]:
data_dir="E:/Data/HIDENIC_EXTRACT_OUTPUT_DIR/POP_SIZE_0/ITEMID_REP/"
data_files=[f for f in os.listdir(data_dir) if 'hospital-overtime-style' in f]

In [ ]:
for target in targets:
    for rep in reps:
        for f in data_files:
            if (target in f) and (rep in f):
                print(target, rep)
                X1=pd.read_hdf(os.path.join(data_dir, f), key='X_train_UPMCPUH')
                df.loc[:, idx[target, :, rep, 'base_rows']] = X1.shape[0]
                df.loc[:, idx[target, :, rep, 'base_cols']] = X1.shape[1]

                for hosp in hospitals:
                    for year in year_range:
                        for month in month_intervals:
                            if ~df.loc[(hosp, year, month), idx[target,:,rep, main_measures]].isna().all():
                                key="X_test_"+hosp+"_"+str(year)+"_"+"-".join([str(i) for i in [month-1, month]])
                                X2=pd.read_hdf(os.path.join(data_dir, f), key)
                                df.loc[(hosp, year, month), idx[target, :, rep, 'data_rows']] = X2.shape[0]
                                df.loc[(hosp, year, month), idx[target, :, rep, 'data_cols']] = X2.shape[1]
                                for indep_test in ["Dcorr"]:
                                    print('indep_test:', indep_test)
                                    t0=time.time()
                                    stat, pvalue = KSample(indep_test).test(X1.values, X2.values)
                                    t1=time.time()
                                    print("runtime=", str((t1-t0)/60), "seconds")
                                    print(hosp, year, month, indep_test, np.round(stat,2), np.round(pvalue,3))
                                    df.loc[(hosp, year, month), idx[target,:,rep, indep_test]] = pvalue

mort_icu raw
indep_test: Dcorr
runtime= 0.2233623425165812 seconds
UPMCEAS 2012 8 Dcorr 0.0 0.0
indep_test: Dcorr
runtime= 0.22421348094940186 seconds
UPMCEAS 2012 10 Dcorr 0.0 0.0
indep_test: Dcorr
runtime= 0.2303679307301839 seconds
UPMCEAS 2013 2 Dcorr 0.0 0.0
indep_test: Dcorr
runtime= 0.23010108470916749 seconds
UPMCEAS 2013 4 Dcorr 0.0 0.0
indep_test: Dcorr
runtime= 0.23210252523422242 seconds
UPMCEAS 2013 10 Dcorr 0.01 0.0
indep_test: Dcorr
runtime= 0.23363700707753499 seconds
UPMCEAS 2013 12 Dcorr 0.0 0.0
indep_test: Dcorr
runtime= 0.22664892673492432 seconds
UPMCEAS 2014 2 Dcorr 0.01 0.0
indep_test: Dcorr
runtime= 0.2212116837501526 seconds
UPMCEAS 2014 4 Dcorr 0.0 0.0
indep_test: Dcorr
runtime= 0.2205608050028483 seconds
UPMCEAS 2014 6 Dcorr 0.01 0.0
indep_test: Dcorr
runtime= 0.2293004790941874 seconds
UPMCEAS 2014 8 Dcorr 0.0 0.0
indep_test: Dcorr
runtime= 0.2258484443028768 seconds
UPMCEAS 2014 10 Dcorr 0.01 0.0
indep_test: Dcorr
runtime= 0.2316856066385905 seconds
UPMCEAS

runtime= 0.2336859146753947 seconds
UPMCMWH 2014 6 Dcorr 0.0 0.006
indep_test: Dcorr
runtime= 0.24417643149693807 seconds
UPMCPAS 2011 2 Dcorr 0.01 0.0
indep_test: Dcorr
runtime= 0.2502808451652527 seconds
UPMCPAS 2011 4 Dcorr 0.02 0.0
indep_test: Dcorr
runtime= 0.23468627532323202 seconds
UPMCPAS 2011 6 Dcorr 0.02 0.0
indep_test: Dcorr
runtime= 0.23567029237747192 seconds
UPMCPAS 2011 8 Dcorr 0.01 0.0
indep_test: Dcorr
runtime= 0.23273483514785767 seconds
UPMCPAS 2011 10 Dcorr 0.01 0.0
indep_test: Dcorr
runtime= 0.23618777990341186 seconds
UPMCPAS 2011 12 Dcorr 0.02 0.0
indep_test: Dcorr
runtime= 0.23018300135930378 seconds
UPMCPAS 2012 2 Dcorr 0.01 0.0
indep_test: Dcorr
runtime= 0.24192482630411785 seconds
UPMCPAS 2012 4 Dcorr 0.02 0.0
indep_test: Dcorr
runtime= 0.24357601006825766 seconds
UPMCPAS 2012 6 Dcorr 0.01 0.0
indep_test: Dcorr
runtime= 0.23992336591084798 seconds
UPMCPAS 2012 8 Dcorr 0.03 0.0
indep_test: Dcorr
runtime= 0.23592050472895304 seconds
UPMCPAS 2012 10 Dcorr 0.02 

In [5]:
df_results = pd.read_pickle("../../../output/HIDENIC_overtime_analysis/hospital_overtime/results_df.pkl")

In [6]:
data_dir="E:/Data/HIDENIC_EXTRACT_OUTPUT_DIR/POP_SIZE_0/ITEMID_REP/"
data_files=[f for f in os.listdir(data_dir) if ('hospital-overtime-style' in f) and ('data_flat' in f)]

In [7]:
data_files

['hospital-overtime-style_UPMCPUH_2008-2009-2010_data_flat_pca_Simple_seed_0_target=los_3.h5',
 'hospital-overtime-style_UPMCPUH_2008-2009-2010_data_flat_pca_Simple_seed_0_target=mort_icu.h5',
 'hospital-overtime-style_UPMCPUH_2008-2009-2010_data_flat_raw_Simple_seed_0_target=los_3.h5',
 'hospital-overtime-style_UPMCPUH_2008-2009-2010_data_flat_raw_Simple_seed_0_target=mort_icu.h5']

In [14]:
out_dir="../../../output/HIDENIC_overtime_analysis/hospital_overtime/"

targets = ['mort_icu', 'los_3']
reps = ['pca', 'raw']
independent_tests = ["CCA", "RV", "MGC", "HHG"]
main_measures = ['AUC', 'APR', 'Acc', 'F1', 'ECE', 'MCE', 'O_E']


hospitals = sorted(site_info["hospital"].unique().tolist())
year_range = np.arange(2011, 2015)
month_step = 2
month_intervals = np.arange(month_step, 13, month_step)

# columns=[]
# for target in targets:
#         for representation in reps:
#             for indep_test in ['base_rows', 'base_cols', 'data_rows', 'data_cols'] + independent_tests:
#                 columns.append((target, representation, indep_test))

# ind=[(hosp, yr, mnth) for hosp in hospitals for yr in year_range for mnth in month_intervals]
# ind=pd.MultiIndex.from_tuples(ind, names=('hospital', 'year', 'month'))
# cols=pd.MultiIndex.from_tuples(columns, names=('target', 'representation', 'indep_test'))
# indep_test_df=pd.DataFrame(index=ind, columns=cols)

In [11]:
indep_test_df = pd.read_pickle(os.path.join(out_dir, "indep_tests.pkl"))

In [ ]:
for indep_test in independent_tests:
    print('indep_test:', indep_test)
    if indep_test=='Dcorr':
        ## I computed this before, skip
        continue
    for target in targets:
        for rep in reps:
            for f in data_files:
                if (target in f) and ("data_flat_"+rep in f):
                    print(target, rep)
                    X1=pd.read_hdf(os.path.join(data_dir, f), key='X_train_UPMCPUH')
                    print('X1_shape:', X1.shape)

                    indep_test_df.loc[:, idx[target, rep, 'base_rows']] = X1.shape[0]
                    indep_test_df.loc[:, idx[target, rep, 'base_cols']] = X1.shape[1]

                    for hosp in hospitals:
                        for year in year_range:
                            for month in month_intervals:
                                if ~df_results.loc[(hosp, year, month), idx[target,:,rep, main_measures]].isna().all():
                                    ## if any non-null measures is available for (hosp, year, month) index
                                    key="X_test_"+hosp+"_"+str(year)+"_"+"-".join([str(i) for i in [month-1, month]])
                                    X2=pd.read_hdf(os.path.join(data_dir, f), key)
                                    print('X2_shape:', X2.shape)
                                    indep_test_df.loc[(hosp, year, month), idx[target, rep, 'data_rows']] = X2.shape[0]
                                    indep_test_df.loc[(hosp, year, month), idx[target, rep, 'data_cols']] = X2.shape[1]
                                    
                                    t0=time.time()
                                    np.random.seed(0)
                                    stat, pvalue = KSample(indep_test).test(X1.values, X2.values, workers=6, auto=True)
                                    t1=time.time()
                                    print("runtime=", str((t1-t0)), "seconds")
                                    print(hosp, year, month, indep_test, np.round(stat,3), np.round(pvalue,3))
                                    
                                    indep_test_df.loc[(hosp, year, month), idx[target, rep, indep_test]] = pvalue
                                    indep_test_df.to_csv(os.path.join(out_dir, "indep_tests.csv"))
                                    indep_test_df.to_pickle(os.path.join(out_dir, "indep_tests.pkl"))
                                    print('#'*60)

indep_test: CCA
mort_icu pca
X1_shape: (7172, 1775)
X2_shape: (104, 1775)
runtime= 196.71182084083557 seconds
UPMCEAS 2012 8 CCA 0.003 0.001
############################################################
X2_shape: (64, 1775)
runtime= 198.3753800392151 seconds
UPMCEAS 2012 10 CCA 0.001 0.003
############################################################
X2_shape: (59, 1775)
runtime= 203.97365498542786 seconds
UPMCEAS 2013 2 CCA 0.004 0.001
############################################################
X2_shape: (50, 1775)
runtime= 197.26373529434204 seconds
UPMCEAS 2013 4 CCA 0.001 0.004
############################################################
X2_shape: (64, 1775)
runtime= 197.06259679794312 seconds
UPMCEAS 2013 10 CCA 0.002 0.001
############################################################
X2_shape: (52, 1775)
runtime= 196.78937911987305 seconds
UPMCEAS 2013 12 CCA 0.002 0.001
############################################################
X2_shape: (55, 1775)
runtime= 196.7465958595276 sec

runtime= 205.16294121742249 seconds
UPMCMCK 2011 12 CCA 0.008 0.001
############################################################
X2_shape: (108, 1775)
runtime= 207.92584824562073 seconds
UPMCMCK 2012 2 CCA 0.009 0.001
############################################################
X2_shape: (119, 1775)
runtime= 236.9067840576172 seconds
UPMCMCK 2012 4 CCA 0.003 0.001
############################################################
X2_shape: (120, 1775)
runtime= 237.0533049106598 seconds
UPMCMCK 2012 6 CCA 0.004 0.001
############################################################
X2_shape: (107, 1775)
runtime= 240.19657564163208 seconds
UPMCMCK 2012 8 CCA 0.005 0.001
############################################################
X2_shape: (93, 1775)
runtime= 244.44064283370972 seconds
UPMCMCK 2012 10 CCA 0.004 0.001
############################################################
X2_shape: (102, 1775)
runtime= 246.66624927520752 seconds
UPMCMCK 2012 12 CCA 0.01 0.012
##################################

runtime= 231.69492530822754 seconds
UPMCPAS 2013 2 CCA 0.006 0.001
############################################################
X2_shape: (179, 1775)
runtime= 230.91035223007202 seconds
UPMCPAS 2013 4 CCA 0.006 0.001
############################################################
X2_shape: (182, 1775)
runtime= 229.22112274169922 seconds
UPMCPAS 2013 6 CCA 0.006 0.001
############################################################
X2_shape: (167, 1775)
runtime= 227.90216040611267 seconds
UPMCPAS 2013 8 CCA 0.005 0.001
############################################################
X2_shape: (152, 1775)
runtime= 229.95465564727783 seconds
UPMCPAS 2013 10 CCA 0.008 0.001
############################################################
X2_shape: (132, 1775)
runtime= 235.53729701042175 seconds
UPMCPAS 2013 12 CCA 0.008 0.001
############################################################
X2_shape: (175, 1775)
runtime= 228.83793544769287 seconds
UPMCPAS 2014 2 CCA 0.009 0.001
###############################

runtime= 200.44229412078857 seconds
UPMCSHY 2014 4 CCA 0.005 0.001
############################################################
X2_shape: (192, 1775)
runtime= 200.14432168006897 seconds
UPMCSHY 2014 6 CCA 0.006 0.001
############################################################
X2_shape: (213, 1775)
runtime= 200.82424354553223 seconds
UPMCSHY 2014 8 CCA 0.006 0.001
############################################################
X2_shape: (226, 1775)
runtime= 201.1595721244812 seconds
UPMCSHY 2014 10 CCA 0.005 0.001
############################################################
X2_shape: (156, 1775)
runtime= 200.35600399971008 seconds
UPMCSHY 2014 12 CCA 0.005 0.001
############################################################
X2_shape: (69, 1775)
runtime= 197.1724922657013 seconds
UPMCSMH 2011 2 CCA 0.005 0.001
############################################################
X2_shape: (74, 1775)
runtime= 197.28970956802368 seconds
UPMCSMH 2011 6 CCA 0.001 0.002
###################################

runtime= 2804.0804936885834 seconds
UPMCHZN 2012 2 CCA 0.013 0.001
############################################################
X2_shape: (87, 7657)
runtime= 2809.5104944705963 seconds
UPMCHZN 2012 4 CCA 0.009 0.001
############################################################
X2_shape: (84, 7657)
runtime= 2811.253610610962 seconds
UPMCHZN 2012 6 CCA 0.01 0.001
############################################################
X2_shape: (83, 7657)
runtime= 2802.55815076828 seconds
UPMCHZN 2012 8 CCA 0.01 0.001
############################################################
X2_shape: (89, 7657)
runtime= 2799.1811623573303 seconds
UPMCHZN 2012 10 CCA 0.01 0.001
############################################################
X2_shape: (79, 7657)
runtime= 2798.140387773514 seconds
UPMCHZN 2012 12 CCA 0.009 0.001
############################################################
X2_shape: (88, 7657)
runtime= 2797.6608850955963 seconds
UPMCHZN 2013 2 CCA 0.009 0.001
############################################

runtime= 2909.4303045272827 seconds
UPMCMER 2013 10 CCA 0.022 0.001
############################################################
X2_shape: (165, 7657)
runtime= 2853.3872604370117 seconds
UPMCMER 2013 12 CCA 0.025 0.001
############################################################
X2_shape: (137, 7657)
runtime= 2852.6371080875397 seconds
UPMCMER 2014 2 CCA 0.024 0.001
############################################################
X2_shape: (186, 7657)
runtime= 2848.732966899872 seconds
UPMCMER 2014 4 CCA 0.03 0.001
############################################################
X2_shape: (152, 7657)
runtime= 2837.0999369621277 seconds
UPMCMER 2014 6 CCA 0.019 0.001
############################################################
X2_shape: (179, 7657)
runtime= 2863.2925295829773 seconds
UPMCMER 2014 8 CCA 0.019 0.001
############################################################
X2_shape: (187, 7657)
runtime= 2853.9306824207306 seconds
UPMCMER 2014 10 CCA 0.021 0.001
################################

runtime= 2930.9723563194275 seconds
UPMCPUH 2014 6 CCA 0.006 0.001
############################################################
X2_shape: (451, 7657)
runtime= 2965.775199174881 seconds
UPMCPUH 2014 8 CCA 0.002 0.001
############################################################
X2_shape: (453, 7657)
runtime= 2945.893723011017 seconds
UPMCPUH 2014 10 CCA 0.008 0.001
############################################################
X2_shape: (314, 7657)
runtime= 2893.5305466651917 seconds
UPMCPUH 2014 12 CCA 0.009 0.001
############################################################
X2_shape: (291, 7657)
runtime= 2889.697262763977 seconds
UPMCSHY 2011 2 CCA 0.017 0.001
############################################################
X2_shape: (309, 7657)
runtime= 2975.668226003647 seconds
UPMCSHY 2011 4 CCA 0.019 0.001
############################################################
X2_shape: (297, 7657)
runtime= 2885.613635778427 seconds
UPMCSHY 2011 6 CCA 0.023 0.001
####################################